In [1]:
from motion.BVH import BVH as BVH
from net.quat import quat
from net.euler import euler
import numpy as np
filename = "./data/Horse/02/run.bvh"
filename2 = "./data/Human/016/16_16.bvh"

horse_skeleton = BVH.load(filename)     
human_skeleton = BVH.load(filename2)

In [2]:
centaur_skeleton = horse_skeleton
centaur_skeleton.root.children[0].children[0].children[0].children[0].children[0] = human_skeleton.root.children[2]

In [3]:
human_skeleton.calculate_global_positions(human_skeleton.root)
horse_skeleton.calculate_global_positions(horse_skeleton.root)
centaur_skeleton.calculate_global_positions(centaur_skeleton.root)

In [30]:
def extract_hierarchy(bvh_file, skeleton):
    for child in skeleton:
        if "End" in child.name:
            bvh_file.write("End Site")
            bvh_file.write("{\n")
            bvh_file.write(f"OFFSET {float(child.offsets[0])} {float(child.offsets[1])} {float(child.offsets[2])}\n")
            bvh_file.write("}\n")
            return
        else:
            bvh_file.write(f"JOINT {child.name}\n")
            bvh_file.write("{\n")
            bvh_file.write(f"OFFSET {float(child.offsets[0])} {float(child.offsets[1])} {float(child.offsets[2])}\n")
            bvh_file.write(f"CHANNELS {len(child.channels)} {child.channels[0]} {child.channels[1]} {child.channels[2]}\n")
            extract_hierarchy(bvh_file, child.children)
            bvh_file.write("}\n")
        
        



In [39]:
def extract_motion(skeleton, frame, frame_data):
    if(len(frame_data) == 0):
        frame_data.append(skeleton.rotations[frame])
    for child in skeleton.children:
        if("End" in child.name):
            return
        frame_data.append(child.rotations[frame])
        extract_motion(child, frame, frame_data)     


In [53]:
import re


def write_bvh_file(filename, skeleton):
    with open(filename, 'w') as bvh_file:
        # Write BVH header
        bvh_file.write("HIERARCHY\n")
        bvh_file.write("ROOT " + skeleton.root.name + "\n")
        bvh_file.write("{\n")

        bvh_file.write(f"OFFSET {float(skeleton.root.offsets[0])} {float(skeleton.root.offsets[1])} {float(skeleton.root.offsets[2])} \n")
        bvh_file.write(f"CHANNELS {len(skeleton.root.channels)} {skeleton.root.channels[0]} {skeleton.root.channels[1]} {skeleton.root.channels[2]} {skeleton.root.channels[3]} {skeleton.root.channels[4]} {skeleton.root.channels[5]}\n")
        
        # Write joint hierarchy
        extract_hierarchy(bvh_file, skeleton.root.children)
        bvh_file.write("}\n")
        
        # Write motion data
        bvh_file.write("MOTION\n")
        bvh_file.write(f"Frames: {int(skeleton.nframes)}\n")
        bvh_file.write(f"Frame Times: {float(skeleton.frame_time)}\n")
        
        frame_data = []

        for frame in range(skeleton.nframes):
            extract_motion(skeleton.root, frame, frame_data)
            frame_data = re.sub('[\[\]]', '', str(frame_data))
            bvh_file.write(f"{str(frame_data)}\n")
            frame_data = []

       



In [54]:
write_bvh_file("test.bvh", centaur_skeleton)